# Test Audio Samples Plotting

In [ ]:
import torchaudio
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import signal
import essentia
import essentia.standard as esstd
import numpy as np
import seaborn as sns
from matplotlib import cm, colors, LogNorm

# %matplotlib notebook
# %matplotlib inline
plt.rcParams['figure.figsize'] = (15, 6) # set plot sizes to something larger than default

In [ ]:
torchaudio.set_audio_backend("sox_io")
sample_rate = 48000

In [ ]:
def check_dc_offset(pred, target):
    pred_offset = pred.mean()
    target_offset = target.mean()
    dc_offset = pred_offset - target_offset

    # Smallest quantization step for 16-bit audio
    quantization_step_16bit = 1 / 32767.0

    # Check if the absolute value of the DC offset is smaller or larger than the quantization step
    if abs(dc_offset) < quantization_step_16bit:
        print("The DC offset is smaller than the smallest quantization step for 16-bit audio.")
    else:
        print("The DC offset is larger than the smallest quantization step for 16-bit audio.")

In [ ]:
def overlap_waveforms(o, t, sample_rate, start, end, title):    

    o_zoom = o[start:end]
    t_zoom = t[start:end]

    # create time vector
    time = range(start, end)

    plt.figure()
    plt.plot(time, o_zoom, alpha=0.8, label="Model")
    plt.plot(time, t_zoom, alpha=0.8, label="Target")
    plt.xlabel("Time (samples)")
    plt.ylabel("Amplitude")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"results/plots/{title}_waves.png")
    plt.show()

In [ ]:
from matplotlib.ticker import ScalarFormatter
import numpy as np
from scipy.signal import spectrogram, get_window
import matplotlib.pyplot as plt

def target_output_spectra(o, t, sample_rate, start, end, title):

    o_zoom = o[start:end]
    t_zoom = t[start:end]

    N = int(len(o_zoom))         # Number of sample points
    T = 1.0 / sample_rate        # sample spacing


    fft_value = 2**14
    overlap = T
    ratio = fft_value // 4
    seg = ratio
    w = get_window('blackman', ratio)


    freq_o, times_o, Sxx_o = spectrogram(
        o_zoom, 
        fs=sample_rate,
        window=w,
        nfft=fft_value, 
        nperseg=seg,
        noverlap=overlap,  
        scaling='spectrum', 
        mode='magnitude'
    )
    freq_t, times_t, Sxx_t = spectrogram(
        t_zoom, 
        fs=sample_rate, 
        nperseg=seg,
        noverlap=overlap,  
        scaling='spectrum', 
        mode='magnitude'
    )

    # Convert magnitude to dB
    o_Sxx_dB = 10 * np.log10(Sxx_o + 1e-10)
    t_Sxx_dB = 10 * np.log10(Sxx_t + 1e-10)

    average_o = np.mean(o_Sxx_dB, axis=1)
    average_t = np.mean(t_Sxx_dB, axis=1)

    plt.figure()

    plt.plot(freq_o, average_o, alpha=1, label="Model", linewidth=0.5)
    plt.plot(freq_t, average_t, alpha=1, label="Target", linewidth=0.5)
    
    plt.grid()
    plt.xscale('log')  # Frequency should often be plotted on a log scale
    plt.xlim([20, 10000])
    plt.title(title)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Average Magnitude (dB)')
    plt.legend()
    
    # Use ScalarFormatter to avoid scientific notation
    ax = plt.gca()
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.xaxis.get_major_formatter().set_scientific(False)
    ax.xaxis.get_major_formatter().set_useOffset(False)
    plt.tight_layout()
    plt.show()


In [ ]:
def process_directory(directory_path, start, end):
    """
    Walk through directory_path, and for each pair of out_{some_model_name}.wav and tgt_{same_model_name}.wav, 
    load them using torchaudio and call overlap_waveforms and two_spectrograms_difference functions.
    """
    directory_path = Path(directory_path)
    output_files = [f for f in directory_path.iterdir() if f.name.startswith("out_") and f.suffix == ".wav"]
    
    for out_file in output_files:
        # Extract the model name and sample rate by splitting on '_'. 
        # This assumes the format is consistent with what's shown.
        parts = out_file.stem.split("_")
        model_name = "_".join(parts[1:-1])  # Combine everything except 'out' and sample rate.
        sample_rate_suffix = parts[-1]
        
        # Construct the expected name for the tgt file
        tgt_file_name = f"tgt_{model_name}_{sample_rate_suffix}.wav"
        tgt_file = directory_path / tgt_file_name
        
        if tgt_file.exists():
            # Load the audio files using torchaudio
            output, sample_rate = torchaudio.load(out_file)
            target, _ = torchaudio.load(tgt_file)
            
            output = output.view(-1).numpy()
            target = target.view(-1).numpy()

            title = model_name + "_" + sample_rate_suffix
            
            # Call your functions
            # overlap_waveforms(output, target, sample_rate, start, end, title)
            # target_output_spectra(output, target, sample_rate, start, end, title)
        else:
            print(f"Matching target file for {out_file} not found!")

In [ ]:
# Test
SAMPLES_DIR = "results/48k/audio/"

start_time = int(sample_rate * 1)
end_time =  int(sample_rate * 3 + 1024)

# process_directory(SAMPLES_DIR, start_time, end_time)

In [ ]:
out_file = "results/48k/audio/out_LSTM-96_48k.wav"
loader = esstd.MonoLoader(filename=out_file)
audio = loader()

In [ ]:
w = esstd.Windowing(type='hann')
spectrum_f = esstd.Spectrum()
logspec_f = esstd.LogSpectrum()

def nth_octave_smoothing(spectrum, n: int = 3):
    N = len(spectrum)
    freq_bins = np.linspace(0, int(sample_rate/2), N)
    y = np.zeros(shape=np.shape(spectrum), dtype = type(spectrum[0]))
    M_1 = len(spectrum) - 1

    for k in range(len(spectrum)):
        a = int(np.round(k * 2 ** (-1 /(2 * n))))
        b = int(np.round(k * 2 ** (1 /(2 * n))))

        if a == b:
            b += 1

        if b > M_1:
            b = M_1

        y[k] = (1 / ((b-1) - a + 1)) * np.sum(spectrum[a:b])
    return y, freq_bins

frame = audio
spec = spectrum_f(w(frame))


log_f_spec, meanT, localT = logspec_f(spec)

y, freq_bins = nth_octave_smoothing(log_f_spec, n=3)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import essentia.standard as esstd

# Load the audio file
loader = esstd.MonoLoader(filename='path_to_audio_file.wav')
audio = loader()
sample_rate = 48000  # adjust based on your file's sample rate

def nth_octave_smoothing(spectrum, n: int = 3):
    N = len(spectrum)
    freq_bins = np.linspace(0, int(sample_rate/2), N)
    y = np.zeros(shape=np.shape(spectrum), dtype = type(spectrum[0]))
    M_1 = len(spectrum) - 1

    for k in range(len(spectrum)):
        a = int(np.round(k * 2 ** (-1 /(2 * n))))
        b = int(np.round(k * 2 ** (1 /(2 * n))))

        if a == b:
            b += 1

        if b > M_1:
            b = M_1

        y[k] = (1 / ((b-1) - a + 1)) * np.sum(spectrum[a:b])
    return y, freq_bins

w = esstd.Windowing(type='hann')
spectrum_function = esstd.Spectrum()
logspec = esstd.LogSpectrum()

frame = audio[1*48000 : 2*48000+1024]
spec = spectrum_function(w(frame))
log_f_spec, meanT, localT = logspec(spec)

y, freq_bins = nth_octave_smoothing(log_f_spec, n=3)

plt.plot(freq_bins, y)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Smoothed Log Spectrum')
plt.show()


In [ ]:
logNorm = esstd.UnaryOperator(type='log')
sns.lineplot(logNorm(mfcc_bands))
plt.title("Log-normalized mel band spectral energies of a frame:")

In [ ]:
# Load the audio file

audio = loader()

# Compute MFCCs
w = es.Windowing(type = 'hann')
spectrum = es.Spectrum()  
mfcc = es.MFCC(numberCoefficients=13)
frame_sz = 1024
hop_sz = 512
mfccs = []


mfccs = essentia.array(mfccs).T  # Transpose for plotting

# Plot MFCCs
plt.figure(figsize=(10, 6))
plt.imshow(mfccs, aspect='auto', origin='lower', cmap='viridis')
plt.ylabel('MFCC Coefficients')
plt.xlabel('Frames')
plt.title('MFCC')
plt.colorbar(label='Amplitude')
plt.tight_layout()
plt.show()
